1. Load the packages

In [ ]:
import os
import torch
import torchvision
import time
from torchvision import models
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.datasets import fetch_openml
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.autograd.function import Function
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from  torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import random

2. Download Office31 dataset

In [ ]:
!wget https://transferlearningdrive.blob.core.windows.net/teamdrive/dataset/office31.zip
!unzip office31.zip

--2022-03-11 08:36:21--  https://transferlearningdrive.blob.core.windows.net/teamdrive/dataset/office31.zip
Resolving transferlearningdrive.blob.core.windows.net (transferlearningdrive.blob.core.windows.net)... 20.150.17.228
Connecting to transferlearningdrive.blob.core.windows.net (transferlearningdrive.blob.core.windows.net)|20.150.17.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79531208 (76M) [application/x-zip-compressed]
Saving to: ‘office31.zip’

office31.zip        100%[===================>]  75.85M  30.0MB/s    in 2.5s    

2022-03-11 08:36:24 (30.0 MB/s) - ‘office31.zip’ saved [79531208/79531208]

Archive:  office31.zip
   creating: office31/
   creating: office31/amazon/
   creating: office31/amazon/back_pack/
  inflating: office31/amazon/back_pack/frame_0001.jpg  
  inflating: office31/amazon/back_pack/frame_0002.jpg  
  inflating: office31/amazon/back_pack/frame_0003.jpg  
  inflating: office31/amazon/back_pack/frame_0004.jpg  
  inflating: 

3. Load and pre-proprecess images

In [ ]:
def load_data(root_path, domain, batch_size, phase):
    transform_dict = {
        'src': transforms.Compose(
        [
         transforms.Resize(224),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
         ]),
        'tar': transforms.Compose(
        [transforms.Resize(224),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
         ])}
    data = datasets.ImageFolder(root=os.path.join(root_path, domain), transform=transform_dict[phase])
    return data

4. Define the function for sampling from the dataset

In [ ]:
def split_class(input_set):
  loop_x=0
  class_indexset=[]
  index=[]
  for i in range(len(input_set)):
    if loop_x==input_set[i][1]:
      index.append(i)
    else:
      class_indexset.append(index)
      index=[]
      index.append(i)
      loop_x=loop_x+1
  class_indexset.append(index)
  return class_indexset

def sample_method(a_list,samplenum):
  random.seed(10) #amend if you want
  if a_list[-1]-a_list[0]>7:
    training_list = random.sample(range(a_list[0], a_list[-1]+1), samplenum)
  else:
    training_list = random.sample(range(a_list[0], a_list[-1]+1), samplenum-1)

  for i in training_list:
    a_list.remove(i)
  return training_list,a_list


def draw_samples(root_path,domain_src,domain_tar,batch_size):
  source = load_data(root_path, domain_src, batch_size, phase='src')
  target = load_data(root_path, domain_tar, batch_size, phase='tar')  
  soucenum=None
  if domain_src=='amazon':
    soucenum=20
  else:
    soucenum=8
  target_number=3
  
  train_x_scr=[]
  train_y_scr=[]

  train_x_tar=[]
  train_y_tar=[]
  test_x_tar=[]
  test_y_tar=[]

  source_indexset=split_class(source)
  target_indexset=split_class(target)

  for each_srclist in source_indexset:
    training_list,testing_list=sample_method(each_srclist,soucenum)
    for i in training_list:
      train_x_scr.append(source[i][0].numpy())
      train_y_scr.append(source[i][1])


  for each_tarlist in target_indexset:
    training_list,testing_list=sample_method(each_tarlist,target_number)

    for i in training_list:
      train_x_tar.append(target[i][0].numpy())
      train_y_tar.append(target[i][1])
    for j in testing_list:
      test_x_tar.append(target[j][0].numpy())
      test_y_tar.append(target[j][1])
  
  return train_x_scr,train_y_scr,train_x_tar,train_y_tar,test_x_tar,test_y_tar
  


5. Self-defined layer for implementing center loss

In [ ]:
class CenterLoss(nn.Module):
    def __init__(self, num_classes, feat_dim, size_average=True):
        super(CenterLoss, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_classes, feat_dim))
        self.centerlossfunc = CenterlossFunc.apply
        self.feat_dim = feat_dim
        self.size_average = size_average

    def forward(self, label, feat):
        batch_size = feat.size(0)
        feat = feat.view(batch_size, -1)
        # To check the dim of centers and features
        if feat.size(1) != self.feat_dim:
            raise ValueError("Center's dim: {0} should be equal to input feature's \
                            dim: {1}".format(self.feat_dim,feat.size(1)))
        batch_size_tensor = feat.new_empty(1).fill_(batch_size if self.size_average else 1)
        loss = self.centerlossfunc(feat, label, self.centers, batch_size_tensor)
        return loss


class CenterlossFunc(Function):
    @staticmethod
    def forward(ctx, feature, label, centers, batch_size):
        ctx.save_for_backward(feature, label, centers, batch_size)
        centers_batch = centers.index_select(0, label.long())
        return (feature - centers_batch).pow(2).sum() / 2.0 / batch_size

    @staticmethod
    def backward(ctx, grad_output):
        feature, label, centers, batch_size = ctx.saved_tensors
        centers_batch = centers.index_select(0, label.long())
        diff = centers_batch - feature
        # init every iteration
        counts = centers.new_ones(centers.size(0))
        ones = centers.new_ones(label.size(0))
        grad_centers = centers.new_zeros(centers.size())

        counts = counts.scatter_add_(0, label.long(), ones)
        grad_centers.scatter_add_(0, label.unsqueeze(1).expand(feature.size()).long(), diff)
        grad_centers = grad_centers/counts.view(-1, 1)
        return - grad_output * diff / batch_size, None, grad_centers / batch_size, None

#Testing function
def main(test_cuda=False):
    print('-'*80)
    device = torch.device("cuda" if test_cuda else "cpu")
    ct = CenterLoss(10,2,size_average=True).to(device)
    y = torch.Tensor([0,0,2,1]).to(device)
    feat = torch.zeros(4,2).to(device).requires_grad_()
    print (list(ct.parameters()))
    
    print (ct.centers.grad)
    out = ct(y,feat)
    print(out.item())
    out.backward()


if __name__ == '__main__':
    torch.manual_seed(999)
    main(test_cuda=False)
    if torch.cuda.is_available():
        main(test_cuda=True)

--------------------------------------------------------------------------------
[Parameter containing:
tensor([[-0.2528,  1.4072],
        [ 0.2910,  1.0365],
        [ 0.6396, -1.0857],
        [-1.6153,  1.5635],
        [ 0.1878, -0.9564],
        [-0.2440, -0.4153],
        [ 0.3259, -1.6059],
        [-0.5272,  0.3401],
        [-1.6526,  0.2108],
        [-1.3302,  0.4676]], requires_grad=True)]
None
0.8543729186058044
--------------------------------------------------------------------------------
[Parameter containing:
tensor([[ 0.9147, -1.1896],
        [-0.7501, -1.5465],
        [-0.4303,  0.3703],
        [ 0.4588,  0.2950],
        [ 0.2126, -0.1098],
        [-0.1349,  2.0038],
        [ 1.8339,  2.3544],
        [ 0.3990, -0.3584],
        [-0.2116,  0.5276],
        [ 0.5683, -0.6786]], device='cuda:0', requires_grad=True)]
None
0.9725638628005981


6. Defined the pre-trained VGG-16 net

In [ ]:
class TransferModel(nn.Module):
    def __init__(self,
                base_model : str = 'VGG16',
                pretrain : bool = True,
                n_class : int = 31):
        super(TransferModel, self).__init__()
        self.base_model = base_model
        self.pretrain = pretrain
        self.n_class = n_class
        if self.base_model == 'VGG16':
            self.model = torchvision.models.vgg16(pretrained=True)
            n_features = self.model.classifier[6].in_features
            fc = torch.nn.Linear(n_features, 1024)
            self.model.classifier[6] = fc
        else:
            pass
        
        self.ip1 = nn.Linear(1024, 128)
        self.ip2 = nn.Linear(128, 31)
        
    def forward(self, x):
        x= self.model(x)
        feature=self.ip1(x)
        feature.view(-1, 128)
        output=self.ip2(feature)
        return feature,output
    def predict(self, x):
        return self.forward(x)

In [ ]:
from torchsummary import summary
RAND_TENSOR = torch.randn(1, 3, 224, 224).cuda()
device = torch.device("cuda")
net = TransferModel().to(device)
output = net(RAND_TENSOR)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

7. Define the funtion for fine-tuning the model pre-trained by ImageNet

In [ ]:
def finetune(model, dataloaders, optimizer,loss_weight):
    centerloss = CenterLoss(31, 128).to(device)
    optimzer4center = torch.optim.SGD(centerloss.parameters(), lr =0.5)
    n_epoch = 100
    criterion = nn.CrossEntropyLoss()
    early_stop = 20
    since = time.time()
    best_acc = 0
    stop = 0
    for epoch in range(0, n_epoch):
        stop += 1
        for phase in ['src']:
            if phase == 'src':
                model.train()
            else:
                model.eval()
            total_loss, total_center,correct = 0, 0,0
            for inputs, labels in dataloaders[phase]:
                inputs, labels= inputs.cuda(),labels.cuda()
                optimizer.zero_grad()
                optimzer4center.zero_grad()
                with torch.set_grad_enabled(phase == 'src'):
                    ip1, outputs = model(inputs)
                    softmax=criterion(outputs, labels)
                    centerlss=centerloss(labels,ip1)
                    loss = softmax + loss_weight * centerlss
                preds = torch.max(outputs, 1)[1]
                if phase == 'src':
                    loss.backward()
                    optimizer.step()
                    optimzer4center.step()
                total_loss += softmax.item() * inputs.size(0)
                total_center +=centerlss.item()
                correct += torch.sum(preds == labels.data)
            epoch_loss = total_loss / len(dataloaders[phase].dataset)
            epoch_acc = correct.double() / len(dataloaders[phase].dataset)
        model.eval()
        correct = 0
        for inputs, labels in dataloaders['tar']:
          inputs, labels= inputs.cuda(),labels.cuda()
          ip1, outputs = model(inputs)
          preds = torch.max(outputs, 1)[1]
          correct += torch.sum(preds == labels.data)
        epoch_acc = correct.double()/ len(dataloaders['tar'].dataset)
        if epoch>90:
            print(f'Epoch: [{epoch:02d}/{n_epoch:02d}]---{phase}, softmax_loss: {epoch_loss:.6f}')
            print_center = total_center / len(dataloaders[phase].dataset)
            print('Center loss:',print_center)
            print('Testing acc:',epoch_acc.item())


8. Define evaluation function

In [ ]:
def trainandevaluation(types, domain_src,domain_tar,random_seed,weight):
  data_folder = 'office31'
  batch_size = 32
  n_class = 31
  random.seed(random_seed) #amend if you want

  train_x_scr,train_y_scr,train_x_tar,train_y_tar,test_x_tar,test_y_tar=draw_samples(data_folder,domain_src,domain_tar,batch_size)


  train_x_scr,train_y_scr,train_x_tar,train_y_tar,test_x_tar,test_y_tar=np.array(train_x_scr),np.array(train_y_scr),np.array(train_x_tar),np.array(train_y_tar),np.array(test_x_tar),np.array(test_y_tar)
  if types=='sourceonly':
    x_train=train_x_scr
    y_train=train_y_scr

  else:
    x_train=np.vstack((train_x_scr,train_x_tar))
    y_train=np.hstack((train_y_scr,train_y_tar))
  
  print("Size of training set:",len(x_train))

  x_test=test_x_tar
  y_test=test_y_tar
  print("Size of testing set:",len(x_test))

  x_train=torch.from_numpy(x_train)
  y_train=torch.from_numpy(y_train)
  x_test=torch.from_numpy(x_test)
  y_test=torch.from_numpy(y_test)

  x_train=x_train.float()
  x_test=x_test.float()
  y_train=y_train.long()
  y_test=y_test.long()

  # form the dataset
  train_dataset=TensorDataset(x_train,y_train)
  val_dataset=TensorDataset(x_test,y_test)

  train_loader= DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
  test_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

  model = TransferModel().cuda()
  param_group = []
  learning_rate = 0.001
  momentum = 5e-3
  for k, v in model.named_parameters():


    if k.__contains__('classifier'):
        # param_group += [{'params': v, 'lr': 10*learning_rate }]
        param_group += [{'params': v, 'lr': 10*learning_rate }]
    elif k.__contains__('ip'):
        param_group += [{'params': v, 'lr': 10*learning_rate }]

    else:
        param_group += [{'params': v, 'lr': learning_rate}]

  print('Types:',types)
  optimizer = torch.optim.SGD(param_group, momentum=momentum)
  dataloaders = {'src': train_loader,
               'tar': test_loader}
  finetune(model, dataloaders, optimizer,weight)

9. Example

Examples below are baseline Models 1, 2, and 3 on the *Amaon to DSLR* domain adaptation task.
You may change the value of variables to get other experimental results.

Note that models will be trained from scratch. Training time should last for around **75 minutes** for one repetition if you use the GPU (GeForce RTX 3090).

Although the results you get may be slightly different from the ones of the manuscript due to randomized initialization, the gap should be small.

In [ ]:
#'amazon', 'webcam','dslr'
types='sourceonly'
source_domain='amazon'
target_domain='dslr'
random_seed=1
loss_weight=0
trainandevaluation(types,source_domain,target_domain,random_seed,loss_weight)

Size of training set: 620
Size of testing set: 407
Types: sourceonly
Epoch: [91/100]---src, softmax_loss: 0.001201
Center loss: 23.01130863312752
Testing acc: 0.6805896805896806
Epoch: [92/100]---src, softmax_loss: 0.000658
Center loss: 22.80469960858745
Testing acc: 0.6781326781326781
Epoch: [93/100]---src, softmax_loss: 0.001555
Center loss: 23.19535620904738
Testing acc: 0.6805896805896806
Epoch: [94/100]---src, softmax_loss: 0.000818
Center loss: 22.833714835874495
Testing acc: 0.6805896805896806
Epoch: [95/100]---src, softmax_loss: 0.001417
Center loss: 22.727716359784527
Testing acc: 0.6707616707616707
Epoch: [96/100]---src, softmax_loss: 0.000681
Center loss: 23.430994735225553
Testing acc: 0.6732186732186732
Epoch: [97/100]---src, softmax_loss: 0.000873
Center loss: 23.58522220734627
Testing acc: 0.6756756756756757
Epoch: [98/100]---src, softmax_loss: 0.001365
Center loss: 23.854941977224044
Testing acc: 0.6683046683046683
Epoch: [99/100]---src, softmax_loss: 0.001344
Center lo

In [ ]:
#'amazon', 'webcam','dslr'
types='softmax'
source_domain='amazon'
target_domain='dslr'
random_seed=1
loss_weight=0
trainandevaluation(types,source_domain,target_domain,random_seed,loss_weight)

Size of training set: 711
Size of testing set: 407
Types: softmax
Epoch: [91/100]---src, softmax_loss: 0.001151
Center loss: 23.89970644751011
Testing acc: 0.8697788697788698
Epoch: [92/100]---src, softmax_loss: 0.001064
Center loss: 23.950973575125264
Testing acc: 0.8697788697788698
Epoch: [93/100]---src, softmax_loss: 0.000473
Center loss: 23.741898548753955
Testing acc: 0.8722358722358722
Epoch: [94/100]---src, softmax_loss: 0.002180
Center loss: 23.486238847134317
Testing acc: 0.8722358722358722
Epoch: [95/100]---src, softmax_loss: 0.000929
Center loss: 23.57689631337355
Testing acc: 0.8673218673218673
Epoch: [96/100]---src, softmax_loss: 0.001531
Center loss: 23.74713615492594
Testing acc: 0.85995085995086
Epoch: [97/100]---src, softmax_loss: 0.000930
Center loss: 24.199723255785205
Testing acc: 0.8648648648648648
Epoch: [98/100]---src, softmax_loss: 0.000560
Center loss: 24.146172245846518
Testing acc: 0.8648648648648648
Epoch: [99/100]---src, softmax_loss: 0.000638
Center loss: 

In [ ]:
#'amazon', 'webcam','dslr'
types='center loss'
source_domain='amazon'
target_domain='dslr'
random_seed=1
loss_weight=0.01
trainandevaluation(types,source_domain,target_domain,random_seed,loss_weight)

Size of training set: 711
Size of testing set: 407
Types: center loss
Epoch: [91/100]---src, softmax_loss: 0.021568
Center loss: 0.2362020998229122
Testing acc: 0.8820638820638821
Epoch: [92/100]---src, softmax_loss: 0.023325
Center loss: 0.23177839633281724
Testing acc: 0.8820638820638821
Epoch: [93/100]---src, softmax_loss: 0.021766
Center loss: 0.23175330973543362
Testing acc: 0.8820638820638821
Epoch: [94/100]---src, softmax_loss: 0.023674
Center loss: 0.2217469181860214
Testing acc: 0.8771498771498771
Epoch: [95/100]---src, softmax_loss: 0.021260
Center loss: 0.2232446905262192
Testing acc: 0.8820638820638821
Epoch: [96/100]---src, softmax_loss: 0.021534
Center loss: 0.22434725070636818
Testing acc: 0.8796068796068796
Epoch: [97/100]---src, softmax_loss: 0.020797
Center loss: 0.21937280633446202
Testing acc: 0.8771498771498771
Epoch: [98/100]---src, softmax_loss: 0.021571
Center loss: 0.22012039713048062
Testing acc: 0.8796068796068796
Epoch: [99/100]---src, softmax_loss: 0.021048